# Using Whisper from OpenAI for ASR of Kathbath Dataset
Dataset: https://asr.iitm.ac.in/Gramvaani/NEW/GV_Eval_3h.tar.gz

## Downloading Whisper from github.

Link: https://github.com/openai/whisper

In [1]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-z6sm1kfi
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-z6sm1kfi
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802826 sha256=481609db22f3c28b40c7098125e654c3ae8b493ef840e1a6c72adc8691cdf1a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-gxxybmi1/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


## Testing audio file

In [2]:
from IPython.display import display, Audio
input_file_path ='/kaggle/input/whisper/GV_Eval_3h/Audio/01-00004-02.mp3'

display(Audio(input_file_path))

 ## Loading ***large-v3*** model

In [3]:
import whisper

model = whisper.load_model('large')

100%|██████████████████████████████████████| 2.88G/2.88G [00:22<00:00, 140MiB/s]


In [4]:
result = model.transcribe(input_file_path,language='hi')
print(result['text'])

 वहीं राहुल कुमार बुगेश कुमार राजा कुमार अनुरख कुमार शमी कुमार रहेश कुमार मधिस कुमार शाहित गर्जेनों कार्यकार भी सामन से जन्मवक दिलपांडे के मौगरा से बिहार मुझवानी लिजर प्रेस्टूर्स


In [9]:
# Creating a pred.txt to store predicted transcripts
pred_file_path = "/kaggle/working/pred.txt"

with open(pred_file_path, 'w') as file:
    pass 

print("Empty text file created:", pred_file_path)

Empty text file created: /kaggle/working/pred.txt


 ## Writing the result in *pred.txt*, doing this in batches to keep track of progress.

In [10]:
import os

input_folder = '/kaggle/input/whisper/GV_Eval_3h/Audio'
text_file_path = '/kaggle/working/pred.txt'

batch_size = 10  

with open(text_file_path, 'w', encoding='utf-8') as text_file:
    
    filenames = sorted(os.listdir(input_folder))
    num_files = len(filenames)
    num_batches = (num_files + batch_size - 1)//batch_size  

    for batch_index in range(num_batches):
        start_index = batch_index * batch_size
        end_index = min((batch_index + 1) * batch_size, num_files)

        print(f"Processing batch {batch_index + 1}/{num_batches}...")
        for filename in filenames[start_index:end_index]:
            if filename.endswith(".mp3"): 
                input_file_path = os.path.join(input_folder, filename)

                result = model.transcribe(input_file_path, language='hi')
                
                #print(result['text'])

               
                text_file.write(f"{os.path.splitext(filename)[0]} {result['text']}\n")

                

Processing batch 1/104...
Processing batch 2/104...
Processing batch 3/104...
Processing batch 4/104...
Processing batch 5/104...
Processing batch 6/104...
Processing batch 7/104...
Processing batch 8/104...
Processing batch 9/104...
Processing batch 10/104...
Processing batch 11/104...
Processing batch 12/104...
Processing batch 13/104...
Processing batch 14/104...
Processing batch 15/104...
Processing batch 16/104...
Processing batch 17/104...
Processing batch 18/104...
Processing batch 19/104...
Processing batch 20/104...
Processing batch 21/104...
Processing batch 22/104...
Processing batch 23/104...
Processing batch 24/104...
Processing batch 25/104...
Processing batch 26/104...
Processing batch 27/104...
Processing batch 28/104...
Processing batch 29/104...
Processing batch 30/104...
Processing batch 31/104...
Processing batch 32/104...
Processing batch 33/104...
Processing batch 34/104...
Processing batch 35/104...
Processing batch 36/104...
Processing batch 37/104...
Processing

### importing result into data frame. 
Notice: some values were conisdered bad_lines, so we lost them. Therefore we are left with 922 rows to compare.

In [30]:
import pandas as pd

df_pred = pd.read_csv("/kaggle/working/pred.txt",on_bad_lines='skip', header=None)
df_pred

0
0    01-00004-02  वहीं राहुल कुमार बुगेश कुमार राजा...
1    01-00008-03  यह बहुत ख्याल जिमेदारी है इसलिए ब...
2    01-00031-03  क्योंकि हमारी पुलिस का फेस्बुक और...
3    01-00078-01  समस्या को सुना जाएगा और महलाओं की...
4    01-00093-01  विकास के लिए सरावन दी एक बड़ा कदम...
..                                                 ...
917  13-00608-02  नमस्कार सरस्था मैं मोबाल मेडिया र...
918  13-00631-03  कि सरकारी सार्यालयों में बढ़ते द्...
919  13-00649-03  नमस्कार सुरुदा मैं धन्बाद मोबाई म...
920  13-00655-01  इसी के साथ बताना चाहता हूँ कि कोई...
921  13-00670-02  इसके साथ 101 होगे के साथ बारा नका...

[922 rows x 1 columns]

In [31]:
df_pred['Title'] = df_pred[0].str[:11]
df_pred['Predicted'] = df_pred[0].str[11:]

df_pred_c = df_pred.copy().drop(columns=[0])
df_pred_c

Title                                          Predicted
0    01-00004-02    वहीं राहुल कुमार बुगेश कुमार राजा कुमार अनुर...
1    01-00008-03    यह बहुत ख्याल जिमेदारी है इसलिए बहुत समय या ...
2    01-00031-03    क्योंकि हमारी पुलिस का फेस्बुक और गुशे सुसल ...
3    01-00078-01    समस्या को सुना जाएगा और महलाओं की समस्या महल...
4    01-00093-01    विकास के लिए सरावन दी एक बड़ा कदम है इस मोटे...
..           ...                                                ...
917  13-00608-02    नमस्कार सरस्था मैं मोबाल मेडिया रिपोर्टर खुश...
918  13-00631-03    कि सरकारी सार्यालयों में बढ़ते द्रफ्ताजार का...
919  13-00649-03    नमस्कार सुरुदा मैं धन्बाद मोबाई मेडिया से रि...
920  13-00655-01    इसी के साथ बताना चाहता हूँ कि कोई ज्ञान नहीं...
921  13-00670-02    इसके साथ 101 होगे के साथ बारा नकाली जाएगी और...

[922 rows x 2 columns]

## Importing text (test file) in a dataframe. 
text file came with the dataset.

In [32]:
df_test = pd.read_csv('/kaggle/input/whisper/GV_Eval_3h/text',header=None)
df_test['Title'] = df_test[0].str[:11]
df_test['Test'] = df_test[0].str[11:]

df_test_c = df_test.copy().drop(columns=[0])
df_test_c

Title                                               Test
0     01-00004-02   वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा ...
1     01-00008-03   युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या ...
2     01-00031-03   क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल म...
3     01-00071-02   नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ...
4     01-00078-01   समस्या को सुना जायेगा और महिलाओं की समस्या मह...
...           ...                                                ...
1027  13-00608-02   नमस्कार श्रोता मैं मोबाइल मिडिया रिपोर्टर खुर...
1028  13-00631-03   की सरकारी कार्यालयों में बढ़ती भ्रष्टाचार का म...
1029  13-00649-03   नमस्कार श्रोताओ में मैं धनबाद मोबाइल मीडिया स...
1030  13-00655-01   इसी के साथ बताना चाहता हूँ कि कोई ध्यान नहीं ...
1031  13-00670-02   इसके साथ एक सौ एक घोड़े के साथ बारात निकाली जा...

[1032 rows x 2 columns]

## Merging both dataframes

In [27]:
df_final = pd.merge(df_test_c, df_pred_c, on='Title')
df_final.reset_index(inplace=True, drop=True)
df_final

Title                                               Test  \
0    01-00004-02   वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा ...   
1    01-00008-03   युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या ...   
2    01-00031-03   क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल म...   
3    01-00078-01   समस्या को सुना जायेगा और महिलाओं की समस्या मह...   
4    01-00093-01   विकास के लिए शराब बंदी एक बड़ा कदम है इस मुद्द...   
..           ...                                                ...   
917  13-00608-02   नमस्कार श्रोता मैं मोबाइल मिडिया रिपोर्टर खुर...   
918  13-00631-03   की सरकारी कार्यालयों में बढ़ती भ्रष्टाचार का म...   
919  13-00649-03   नमस्कार श्रोताओ में मैं धनबाद मोबाइल मीडिया स...   
920  13-00655-01   इसी के साथ बताना चाहता हूँ कि कोई ध्यान नहीं ...   
921  13-00670-02   इसके साथ एक सौ एक घोड़े के साथ बारात निकाली जा...   

                                             Predicted  
0      वहीं राहुल कुमार बुगेश कुमार राजा कुमार अनुर...  
1      यह बहुत ख्याल जिमेदारी है इसलिए बहुत समय या ...  
2      क्योंकि हमारी पुलिस का फेस्बुक और गुशे सुसल ...  
3      समस्या को सुना जाएगा और महलाओं की समस्या महल...  
4      विकास के लिए सरावन दी एक बड़ा कदम है इस मोटे...  
..                                                 ...  
917    नमस्कार सरस्था मैं मोबाल मेडिया रिपोर्टर खुश...  
918    कि सरकारी सार्यालयों में बढ़ते द्रफ्ताजार का...  
919    नमस्कार सुरुदा मैं धन्बाद मोबाई मेडिया से रि...  
920    इसी के साथ बताना चाहता हूँ कि कोई ज्ञान नहीं...  
921    इसके साथ 101 होगे के साथ बारा नकाली जाएगी और...  

[922 rows x 3 columns]

## Using Jiwer for WER calculation.

In [24]:
! pip install jiwer

In [25]:
import jiwer

wer = jiwer.wer(list(df_final["Test"]), list(df_final["Predicted"]))
print(f"WER: {wer * 100:.2f} %")

WER: 52.73 %
